In [1]:
import pickle
import pandas
import numpy as np

import keras
from keras.preprocessing import image
from keras import applications
from keras.models import Sequential
from keras.applications import vgg16
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
from keras.layers import Conv2D, Conv3D,Input, ZeroPadding3D, Reshape,LSTM
from keras.layers.convolutional import Convolution2D, Convolution3D, MaxPooling2D, ZeroPadding2D,ZeroPadding3D 
from keras.layers.core import Reshape
import os
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.callbacks import CSVLogger
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.optimizers import SGD
from keras.layers import Merge 
import numpy as np
import keras
import keras.backend as K
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LambdaCallback
from keras import regularizers
from keras import backend as K
from keras.models import load_model
from sklearn.metrics import mean_squared_error

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [15]:
X = np.load("train_X_normalized_lstm_delta.pkl.npy")
Y = np.load("train_Y_normalized_lstm_delta.pkl.npy")
X_test = np.load("test_X_normalized_lstm_delta.pkl.npy")
Y_test = np.load("test_Y_normalized_lstm_delta.pkl.npy")
reg_Y = np.load("test_Y_normalized_lstm_regression_v2.pkl.npy")
# X = np.load("train_X_normalized_lstm_classification.pkl.npy")
# Y = np.load("train_Y_normalized_lstm_classification.pkl.npy")

In [16]:
X.shape

(622641, 60, 5)

In [17]:
X = X.reshape(X.shape[0], X.shape[1], -1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], -1)

In [18]:
Y.shape

(622641, 8)

In [19]:
def mse_pred(y_true,y_pred):
    categories = [-0.15, -0.075, -0.03,-0.005,0.005,0.03,0.075,0.15]
    for i in len(y_true):
        pred = categories[np.argmax(y_pred[i])]
        se += (y_true - pred) ** 2
    return se/len(y_true)


In [24]:
def build_model(X):
    model = Sequential()

    model.add(LSTM(512,
        input_shape=(X.shape[1],X.shape[2]),
        return_sequences=False))
    model.add(Dropout(0.2))

#     model.add(LSTM(
#         512,
#         return_sequences=False))
#     model.add(Dropout(0.2))

    model.add(Dense(8, activation = 'softmax')) ############ change dense layer node number to number of baskets.
    #Adm = keras.optimizers.Adam(lr = 0.001)
    model.compile(loss="categorical_crossentropy", optimizer='sgd', metrics=['categorical_accuracy'])
    #model.compile(loss="categorical_crossentropy", optimizer='sgd', metrics=['accuracy'])
    # K.tensorflow_backend._get_available_gpus()
    return model


In [25]:
model = build_model(X)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 512)               1060864   
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 4104      
Total params: 1,064,968
Trainable params: 1,064,968
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# load previous model
#model = load_model('lstm_delta.h5')
n = 1 # number of epochs before test

for e in range(1):
    print("Epoch ",e,"to",e+n)
    # train
    res = model.fit(X, Y,batch_size=512, epochs=n,verbose=1, shuffle=True,validation_split = 0.1)

    # evaluate on test set, print [loss, accuracy]
    test_result = model.evaluate(X_test, Y_test)
    print(test_result)
    
    # evaluate on test set again, print equivalent
    Y_pred = model.predict(X_test)
    mse = mse_pred(reg_Y,Y_pred)
    print("mean squared error equivalent in test set is", mse)
    

Epoch  0 to 1
Train on 560376 samples, validate on 62265 samples
Epoch 1/1
 28160/560376 [>.............................] - ETA: 4:44 - loss: 2.0484 - categorical_accuracy: 0.2733

In [ ]:
#model.save("lstm_delta.h5")
Y_pred = model.predict(X_test)
mse = mse_pred(reg_Y,Y_pred)

In [ ]:
print(mse)